In [144]:
import requests
from bs4 import BeautifulSoup
import pyodbc

url = 'https://www.starbucks.com.tw/products/drinks/view.jspx?catId=1'

try:
    # 發送GET請求
    response = requests.get(url)
    response.raise_for_status()  # 檢查是否成功獲取頁面

    # 使用BeautifulSoup解析HTML內容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 找到所有的文章區塊
    articles = soup.find_all('article')

    drinks = []

    for article in articles:
        # 獲取文章標題
        title = article.find('h2').text.strip()

        # 只處理熱濃縮咖啡飲料的部分
        if title == '熱濃縮咖啡飲料':
            # 找到飲料列表
            drinks_list = article.find('ul').find_all('li')

            for drink_item in drinks_list:
                # 獲取中文名和英文名
                title_cn = drink_item.find('h1', class_='title_cn').text.strip()
                title_en = drink_item.find('h3', class_='title_en').text.strip()

                # 將飲料資訊添加到列表中
                drinks.append((title_cn, title_en))

except requests.exceptions.RequestException as e:
    # 處理請求異常
    print(f'無法取得網頁內容: {e}')
except AttributeError as e:
    # 處理屬性錯誤
    print(f'解析網頁內容時出錯: {e}')

# 印出所有熱濃縮咖啡飲料的資訊
for drink in drinks:
    print(f'中文名稱: {drink[0]}')
    print(f'英文名稱: {drink[1]}')
    print('---------------------')

# 資料庫連接字串設定
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=WEIWEI;"  # 替換為您的伺服器名稱
    "DATABASE=master;"  # 替換為您的資料庫名稱
    "UID=wei;"
    "PWD=132476;"
    "TrustServerCertificate=yes;"
)

conn = None  # 初始化連線變數

try:
    # 建立與資料庫的連線
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()

    # 創建表格（如果不存在）
    cursor.execute("""
    IF OBJECT_ID('drinks', 'U') IS NULL
    CREATE TABLE drinks (
        id INT IDENTITY(1,1) PRIMARY KEY,
        title_cn NVARCHAR(100),
        title_en NVARCHAR(100)
    )
    """)
    conn.commit()

    # 插入資料
    insert_query = """
    INSERT INTO drinks (title_cn, title_en)
    VALUES (?, ?)
    """
    for drink in drinks:
        cursor.execute(insert_query, drink)
        conn.commit()  # 確保每次插入後都提交事務
        print(f'已插入資料: {drink[0]}, {drink[1]}')

    print('所有資料插入成功')

except pyodbc.Error as e:
    print(f'資料庫操作錯誤: {e}')

finally:
    if conn:
        conn.close()


中文名稱: 芝麻瑪奇朵
英文名稱: Sesame Macchiato
---------------------
中文名稱: 可可瑪奇朵
英文名稱: Cocoa Macchiato
---------------------
中文名稱: 那堤
英文名稱: Caffè  Latte
---------------------
中文名稱: 焦糖瑪奇朵
英文名稱: Caramel Macchiato
---------------------
中文名稱: 摩卡
英文名稱: Caffè Mocha
---------------------
中文名稱: 卡布奇諾
英文名稱: Cappuccino
---------------------
中文名稱: 美式咖啡
英文名稱: Caffè Americano
---------------------
中文名稱: 濃縮咖啡
英文名稱: Espresso
---------------------
中文名稱: 特選馥郁那堤
英文名稱: Espresso Choice Extra Shot Latte
---------------------
中文名稱: 馥列白
英文名稱: Flat White
---------------------
已插入資料: 芝麻瑪奇朵, Sesame Macchiato
已插入資料: 可可瑪奇朵, Cocoa Macchiato
已插入資料: 那堤, Caffè  Latte
已插入資料: 焦糖瑪奇朵, Caramel Macchiato
已插入資料: 摩卡, Caffè Mocha
已插入資料: 卡布奇諾, Cappuccino
已插入資料: 美式咖啡, Caffè Americano
已插入資料: 濃縮咖啡, Espresso
已插入資料: 特選馥郁那堤, Espresso Choice Extra Shot Latte
已插入資料: 馥列白, Flat White
所有資料插入成功


In [1]:
from flask import Flask, render_template
import pyodbc
app = Flask(__name__)
@app.route('/')
def index():
    conn_str = (
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=WEIWEI;"  # Replace with your server name
        "DATABASE=master;"  # Replace with your database name
        "UID=wei;"
        "PWD=132476;"
        "TrustServerCertificate=yes;"
    )

    conn = None
    drinks = []

    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()

        # Query the drinks from the database
        cursor.execute("SELECT title_cn, title_en FROM drinks")
        rows = cursor.fetchall()

        for row in rows:
            drinks.append({
                'title_cn': row.title_cn,
                'title_en': row.title_en
            })

    except pyodbc.Error as e:
        print(f'Database error: {e}')

    finally:
        if conn:
            conn.close()

    return render_template('index.html', drinks=drinks)
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Jun/2024 20:10:07] "GET / HTTP/1.1" 200 -
